Plotting 5x6 images of spectra

In [ ]:
#get spectra in desired redshift range
redshifts = [1.1, 1.2]

#Open the catalog of ~750k QSOs
hdul = fits.open("DR16Q_v4.fits")

data = hdul[1].data

#Pick out some columns that might be useful
#Need to convert data from big- to little-endian for pandas

d = {
    "names" : data['SDSS_NAME'].byteswap().newbyteorder(),
    "plate" : data['PLATE'].byteswap().newbyteorder(),
    "mjd"   : data['MJD'].byteswap().newbyteorder(),
    "fiber" : data['FIBERID'].byteswap().newbyteorder(),
    "z"     : data['Z'].byteswap().newbyteorder(),             #best identified redshift 
    "z_vi"  : data['Z_VI'].byteswap().newbyteorder(),          #redshift from visual inspection
    "z_src" : data['SOURCE_Z'].byteswap().newbyteorder(),      #source of best-redshift determination
    "S2N"   : data['SN_MEDIAN_ALL'].byteswap().newbyteorder(), #median S/N of pixels (rough way to sort)
    "BAL"   : data["BAL_PROB"].byteswap().newbyteorder()
}

#sort by redshift
df = pd.DataFrame(data=d)
df.to_csv("quasars.csv")
df.sort_values("z", inplace=True)
df.shape, df[df["z"]==0.00166803679894656]

#Take this redshift range for this notebook, then sort by S/N
dframe = df[((df["z"]>redshifts[0])&(df["z"]<redshifts[1]))]
dframe.sort_values("S2N", ascending=False, inplace=True)
len(dframe)

(dframe["BAL"]<=0.).sum()
dframe["names"].iloc[0]

In [3]:
#plotting helper functions & variables
import numpy as np, matplotlib.pyplot as plt

def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 0))  # outward by 0 points
        else:
            spine.set_color('none')  # don't draw spine

    # turn off ticks where there is no spine
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        # no yaxis ticks
        ax.yaxis.set_ticks([])

    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        # no xaxis ticks
        ax.xaxis.set_ticks([])

from SDSSRM_11_Pix_Filter_Trevor import pixel_filter
plt.rcParams['axes.xmargin'] = 0 #turn off whitespace in plots

N = 30 #number to plot on a page 
FNF = np.array([]) #save 'missing' spectra
DPI = 15 #resolution of plots

NameError: name 'plt' is not defined

In [1]:
#A z=0.24 quasar with decent S2N
#spec_bad = fits.open("lite/7751/spec-7751-58081-0008.fits")
spec_bad = fits.open("spec-7751-58081-0008.fits")

loglam_bad = spec_bad[1].data["LOGLAM"]
wave_bad   = 10.**loglam_bad
mask_bad   = ((wave_bad>7500)&(wave_bad<9000))
wave_bad   = wave_bad[mask_bad]
#wave_bad  /= spec_bad[1].data["Z"] #need to change
flux_bad   = spec_bad[1].data["FLUX"][mask_bad]

#A z=1.12 quasar
#spec_good = fits.open("lite/3861/spec-3861-55274-0422.fits")
spec_good = fits.open("spec-3861-55274-0422.fits")

loglam_good = spec_good[1].data["LOGLAM"]
wave_good   = 10.**loglam_good
mask_good   = ((wave_good>3778)&(wave_good<6245))
wave_good   = wave_good[mask_good]
#wave_good  /= spec_good[1].data["Z"]
flux_good   = spec_good[1].data["FLUX"][mask_good]



NameError: name 'fits' is not defined

In [ ]:
#plot just the good spectra

fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(42,40), frameon=False, dpi=DPI)   #dpi good for degrading resolution of plots
plt.setp(axs, xticks=[], yticks=[])                      #Don't want any ticks or labels

if os.path.exists("plot_names.txt"):
    mode = 'a'
else:
    mode = 'w'
    
with open("plot_names.txt", mode) as f:

    row, col = 0, 0
    for i in range(1, N+1):
        axs[row, col].plot(wave_good, flux_good, color="k")
        adjust_spines(axs[row, col], [])

        #increment col every time we plot
        col += 1

        #if col number is 6 it means we are on first col of next row
        if i%6 == 0:
            row += 1
            col = 0
        
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0) #remove whitespace between subplots
    plt.savefig("stack30_same_lowres.png")
    #plt.savefig("stack30_same_highres.png") 
    plt.show()
    
    f.write("stack30_same_lowres.png")

In [2]:
#now insert a bad spectra

#number of images with a bad spectra
n_bad = N

if os.path.exists("plot_names.txt"):
    mode = 'a'
else:
    mode = 'w'
    
with open("plot_names.txt", mode) as f:
    
    for k in range(1, n_bad+1):
        fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(42,40), frameon=False, dpi=DPI)   #dpi good for degrading resolution of plots
        plt.setp(axs, xticks=[], yticks=[])                      #Don't want any ticks or labels

        row, col = 0,0
        for i in range(1, N+1):
            if i==k:
                axs[row, col].plot(wave_bad, flux_bad, color="k")
            else:
                axs[row, col].plot(wave_good, flux_good, color="k")

            adjust_spines(axs[row, col], [])

             #increment col every time we plot
            col += 1

            #if col number is 6 it means we are on first col of next row
            if i%6 == 0:
                row += 1
                col = 0

        plt.tight_layout()
        plt.subplots_adjust(wspace=0, hspace=0)   #remove whitespace between subplots
        #plt.savefig("spectra30_1diff_highres/stack30_1diff_highres_%s.png" % str(k+1))
        plt.savefig("stack30_1diff_lowres_%s.png" % str(k))
        
        f.write("stack30_1diff_lowres_%s.png\n" % str(k))



NameError: name 'N' is not defined

In [ ]:
#now insert 2 bad spectra

n_bad = 100

if os.path.exists("plot_names.txt"):
    mode = 'a'
else:
    mode = 'w'
    
with open("plot_names.txt", mode) as f:

    for k in range(1, n_bad+1):
        fig, axs = plt.subplots(nrows=5, ncols=6, figsize=(42,40), frameon=False, dpi=DPI)   #dpi good for degrading resolution of plots
        plt.setp(axs, xticks=[], yticks=[])                      #Don't want any ticks or labels

        inds = np.random.randint(0, N, 2)

        row, col = 0, 0
        for i in range(1, N+1):
            if i in inds:
                axs[row, col].plot(wave_bad, flux_bad, color="k")
            else:
                axs[row, col].plot(wave_good, flux_good, color="k")

            adjust_spines(axs[row, col], [])

            #increment col every time we plot
            col += 1

            #if col number is 6 it means we are on first col of next row
            if i%6 == 0:
                row += 1
                col = 0

        plt.tight_layout()
        plt.subplots_adjust(wspace=0, hspace=0)   #remove whitespace between subplots
        #plt.savefig("spectra30_2randdiff_highres/stack30_2diff_highres_%d.png" % k)
        plt.savefig("stack30_2diff_lowres_%d.png" % k)
        
        f.write("stack30_2diff_lowres_%d.png\n" % k)